<a href="https://colab.research.google.com/github/catburger1337/paul_allens_folio/blob/main/QxQ_Capstone_Quantum_Encrypt_QKD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
**Capstone Project - Quantum encryption in Python with Quantum Key Distribution (QKD)**
---
Message: "*Lets eat some chocolate ice cream*"

In [ ]:
#@title
!pip install cirq --quiet
import cirq
import math
import binascii
import numpy as np
from random import choices
def binary_labels(num_qubits):
    return [bin(x)[2:].zfill(num_qubits) for x in range(2 ** num_qubits)]

In [ ]:
message = "type here"
#Type your message here inside quotation marks.

In [ ]:
encode_gates = {0: cirq.I, 1: cirq.X}
basis_gates = {'Z': cirq.I, 'X': cirq.H}


---
**key_size(message)**
---

In [ ]:
def key_size(message):
  binary_string = " ".join(f"{ord(i):08b}" for i in message)

  count_ones = binary_string.count("1")
  count_zeros = binary_string.count("0")
  key_size.total_values = count_ones + count_zeros

  key_size.num_bits = key_size.total_values / math.exp(math.log10(key_size.total_values))
  key_size.num_bits = math.ceil(key_size.num_bits)

  print('\nTotal of numbers coded in binary:', key_size.total_values)
  print('\nThe generated key will have', key_size.num_bits, 'bits')

In [ ]:
key_size(message)

In [ ]:
key_size.num_bits
qubits = cirq.NamedQubit.range(key_size.num_bits, prefix = 'q')
qubits

---
**encryption()**
---

In [ ]:
def encryption():
  encryption.sender_key = choices([0, 1], k = key_size.num_bits)
  encryption.sender_bases = choices(['Z', 'X'], k = key_size.num_bits)
  encryption.sender_circuit = cirq.Circuit()

  for bit in range(key_size.num_bits):

    encode_value = encryption.sender_key[bit]
    encode_gate = encode_gates[encode_value]

    basis_value = encryption.sender_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    encryption.sender_circuit.append(encode_gate(qubit))
    encryption.sender_circuit.append(basis_gate(qubit))

  print('\nAlice\'s randomly chosen bases: ', encryption.sender_bases)
  print('\nAlice\'s initial key: ', encryption.sender_key)
  print('\nAlice\'s Phase 1 circuit:\n', encryption.sender_circuit)

In [ ]:
encryption()

---
**decryption()**
---

In [ ]:
def decryption():
  decryption.receiver_bases = choices(['Z', 'X'], k = key_size.num_bits)
  receiver_circuit = cirq.Circuit()

  for bit in range(key_size.num_bits):

    basis_value = decryption.receiver_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    receiver_circuit.append(basis_gate(qubit))

  receiver_circuit.append(cirq.measure(qubits, key = 'receiver key'))

  bb84_circuit = encryption.sender_circuit + receiver_circuit

  sim = cirq.Simulator()
  results = sim.run(bb84_circuit)
  decryption.receiver_key = results.measurements['receiver key'][0]


  print('Bob\'s randomly chosen bases: ', decryption.receiver_bases)
  print('Bob\'s initial key: ', decryption.receiver_key)
  print('Bob\'s Phase 2 circuit:\n', receiver_circuit)

In [ ]:
decryption()

---
**Comparision of keys and reveal of message**
---

In [ ]:
final_sender_key = []
final_receiver_key = []

for bit in range(key_size.num_bits):

  if encryption.sender_bases[bit] == decryption.receiver_bases[bit]:
    final_sender_key.append(encryption.sender_key[bit])
    final_receiver_key.append(decryption.receiver_key[bit])

if final_sender_key[0] == final_receiver_key[0]:
  final_alice_key = final_sender_key[1:]
  final_bob_key = final_receiver_key[1:]


  print('\nAlice\'s Key: ', final_sender_key)
  print('\nBob\'s Key: ', final_receiver_key)
  print('\nWe can use our keys!')
  print('\nMessage sent: ', message)

else:
  print('\n\nWe have a problem. Eve was listening.')

---
**END**
---